In [2]:
import azureml.core
from azureml.core import Experiment, Workspace, Dataset, Datastore, ScriptRunConfig
import os
import shutil

# check core SDK version number

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.18.0


In [4]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
# get metadata about the workspace
print(ws.name, ws.location, ws.resource_group, sep='\t')
# list the registered datastores
ws.datastores

solar-compviz	eastus2	PLACE


{'solardatafilestore': {
   "name": "solardatafilestore",
   "container_name": "gee-data",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceblobstore': {
   "name": "workspaceblobstore",
   "container_name": "azureml-blobstore-20200d5f-4836-4a79-bf1c-7bd46e0d0c35",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspacefilestore': {
   "name": "workspacefilestore",
   "container_name": "azureml-filestore-20200d5f-4836-4a79-bf1c-7bd46e0d0c35",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 }}

In [4]:
# register our data share containing image data in this workspace
Datastore.register_azure_file_share(
    account_key = 'andMlwgRrZ68ybpJ3d24F9pffr+tqPxlnLxM6qqB8gD75BvMrQKr/6IbvUpRy8cBPytynlLoc78iN2TFU7rWPA==',
    workspace = ws,
    datastore_name = 'solardatafilestore',
    file_share_name = 'gee-data',
    account_name = 'solarcompviz6161005396')

{
  "name": "solardatafilestore",
  "container_name": "gee-data",
  "account_name": "solarcompviz6161005396",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [5]:
# create a file dataset that can be used in training
datastore = Datastore.get(workspace = ws, datastore_name = 'solardatafilestore')
datastore_paths = [(datastore, 'CPK_solar'), (datastore, 'NC_solar')]
cpk_dataset = Dataset.File.from_files(path = datastore_paths[0])
nc_dataset = Dataset.File.from_files(path = datastore_paths[1])

cpk_dataset = cpk_dataset.register(
    workspace=ws,
    name='gee-cpk-solar-data',
    description='training and eval TFRecords for solar arrays exported from GEE',
    create_new_version=True)

nc_dataset = nc_dataset.register(
    workspace=ws,
    name='gee-nc-solar-data',
    description='training and eval TFRecords for solar arrays exported from GEE',
    create_new_version=True)